# Code and Equations for a 3-Layer Neural Network

In [1]:
import torch
from torch import Tensor
from typing import Tuple

In [2]:
# You can ignore these functions. I used them to generate
# all of the matrices. I left them in for those who are
# into that sort of thing.


def print_bvector(template, row):
    row_minus_one = str(row - 1) if isinstance(row, int) else f"{row}-1"
    print("\\begin{bmatrix}")
    print(f"{template.replace('row', '1')} \\\\")
    print(f"{template.replace('row', '2')} \\\\")
    print("\\vdots \\\\")
    print(f"{template.replace('row', row_minus_one)} \\\\")
    print(f"{template.replace('row', str(row))} \\\\")
    print("\\end{bmatrix}")


def print_bvector_row(template, col):
    col_minus_one = str(col - 1) if isinstance(col, int) else f"{col}-1"
    print("\\begin{bmatrix}")
    print(f"{template.replace('col', '1')} &")
    print(f"{template.replace('col', '2')} &")
    print("\\cdots &")
    print(f"{template.replace('col', col_minus_one)} &")
    print(f"{template.replace('col', str(col))}")
    print("\\end{bmatrix}")


def print_bmatrix(template, row, col, compress=False):

    row_minus_one = str(row - 1) if isinstance(row, int) else f"{row}-1"
    col_minus_one = str(col - 1) if isinstance(col, int) else f"{col}-1"

    print("\\begin{bmatrix}")
    # Row 1
    c1 = template.replace("col", "1").replace("row", "1")
    c2 = template.replace("col", "2").replace("row", "1")
    c3 = template.replace("col", col_minus_one).replace("row", "1")
    c4 = template.replace("col", str(col)).replace("row", "1")
    if compress:
        print(f"{c1} & {c2} & \\cdots & {c4}\\\\")
    else:
        print(f"{c1} & {c2} & \\cdots & {c3} & {c4}\\\\")
    c1 = template.replace("col", "1").replace("row", "2")
    c2 = template.replace("col", "2").replace("row", "2")
    c3 = template.replace("col", col_minus_one).replace("row", "2")
    c4 = template.replace("col", str(col)).replace("row", "2")
    if compress:
        print(f"{c1} & {c2} & \\cdots & {c4}\\\\")
        print("\\vdots & \\vdots & \ddots & \\vdots \\\\")
    else:
        print(f"{c1} & {c2} & \\cdots & {c3} & {c4}\\\\")
        print("\\vdots & \\vdots & \ddots & \\vdots & \\vdots \\\\")
    c1 = template.replace("col", "1").replace("row", row_minus_one)
    c2 = template.replace("col", "2").replace("row", row_minus_one)
    c3 = template.replace("col", col_minus_one).replace("row", row_minus_one)
    c4 = template.replace("col", str(col)).replace("row", row_minus_one)
    if not compress:
        print(f"{c1} & {c2} & \\cdots & {c3} & {c4}\\\\")
    c1 = template.replace("col", "1").replace("row", str(row))
    c2 = template.replace("col", "2").replace("row", str(row))
    c3 = template.replace("col", col_minus_one).replace("row", str(row))
    c4 = template.replace("col", str(col)).replace("row", str(row))
    if compress:
        print(f"{c1} & {c2} & \\cdots & {c4}\\\\")
    else:
        print(f"{c1} & {c2} & \\cdots & {c3} & {c4}\\\\")
    print("\\end{bmatrix}")


# print_bvector("x^{(i)}_{row}", "n_x")
# print_bvector("image^{(i)}_{pixel_{row}}", "n_x")
# print_bmatrix("x^{(col)}_{row}", "n_x", "m")
# print_bmatrix("y^{(col)}_{row}", "n_y", "m")
# print_bmatrix("x^{(col)}_{row}", 784, 60000)
# print_bmatrix("y^{(col)}_{row}", 10, 60000)
# print_bvector_row("y^{(col)}", 60000)
# print_bmatrix("w^{[l]}_{row,col}", "n_l", "n_{l-1}")
# print_bvector("b^{[l]}_{row}", "n_l")

# Computing Z
# print_bmatrix("w^{[l]}_{row,col}", "n_l", "n_{l-1}", True)
# print_bmatrix("A^{(col)[l-1]}_{row}", "n_{l-1}", "m", True)
# print_bmatrix("w^{[l]}_{row} \cdot a^{(col)[l-1]} + b^{[l]}_{row}", "n_l", "m")

# And A
# print_bmatrix("g(w^{[l]}_{row} \cdot a^{(col)[l-1]} + b^{[l]}_{row})", "n_l", "m")

## The Dataset

$$
\mathcal{D} = \{X, Y\}
$$

Where $D$ is the dataset and it comprises input features $X$ and output targets $Y$.

The input features $X$ is a matrix containing all features of all input examples. **Let's use MNIST as our working example.** We represent a single image $x^{(i)}$ as a *column* vector:

$$
x^{(i)} =
\begin{bmatrix}
x^{(i)}_{1} \\
x^{(i)}_{2} \\
\vdots \\
x^{(i)}_{n_x-1} \\
x^{(i)}_{n_x} \\
\end{bmatrix}
=
\begin{bmatrix}
image^{(i)}_{pixel_{1}} \\
image^{(i)}_{pixel_{2}} \\
\vdots \\
image^{(i)}_{pixel_{n_x-1}} \\
image^{(i)}_{pixel_{n_x}} \\
\end{bmatrix}
$$

where $n_x$ is the number of input features for a single example. For MNIST, $n_x = 28\cdot28 = 784$ (and for our neural network, we'll say $n_0=n_x$).

Each column in $X$ is a single input instance (called an example or sample), and when you stack all $m$ examples side-by-side, you end up with $X$.

$$
X = 
\begin{bmatrix}
x^{(1)}_{1} & x^{(2)}_{1} & \cdots & x^{(m-1)}_{1} & x^{(m)}_{1}\\
x^{(1)}_{2} & x^{(2)}_{2} & \cdots & x^{(m-1)}_{2} & x^{(m)}_{2}\\
\vdots & \vdots & \ddots & \vdots & \vdots \\
x^{(1)}_{n_x-1} & x^{(2)}_{n_x-1} & \cdots & x^{(m-1)}_{n_x-1} & x^{(m)}_{n_x-1}\\
x^{(1)}_{n_x} & x^{(2)}_{n_x} & \cdots & x^{(m-1)}_{n_x} & x^{(m)}_{n_x}\\
\end{bmatrix}
$$

We say that $x^{(i)} \in \mathcal{R}^{n_x}$ (each input example is $n_x$ real values) and $X \in \mathcal{R}^{n_x \times m}$ (the entire input is a $(n_x, m)$ matrix).

The tagets (aka labels) are in the matrix $Y$. For MNIST, $Y$ will be a $(1, m)$ matrix, but in general, $Y$ is $(n_y, m)$. **Note, I've been a bit inconsistent about the shape of $Y$ on diagrams and in code. From here out, I will follow this notation.**

$$
Y = 
\begin{bmatrix}
y^{(1)}_{1} & y^{(2)}_{1} & \cdots & y^{(m-1)}_{1} & y^{(m)}_{1}\\
y^{(1)}_{2} & y^{(2)}_{2} & \cdots & y^{(m-1)}_{2} & y^{(m)}_{2}\\
\vdots & \vdots & \ddots & \vdots & \vdots \\
y^{(1)}_{n_y-1} & y^{(2)}_{n_y-1} & \cdots & y^{(m-1)}_{n_y-1} & y^{(m)}_{n_y-1}\\
y^{(1)}_{n_y} & y^{(2)}_{n_y} & \cdots & y^{(m-1)}_{n_y} & y^{(m)}_{n_y}\\
\end{bmatrix}
$$

We say that $y^{(i)} \in \mathcal{R}^{n_y}$ (each target is $n_y$ real values) and $Y \in \mathcal{R}^{n_y \times m}$ (the entire input is a $(n_y, m)$ matrix). For classification tasks, each $y^{(i)}_j$ (each value for each target) is usually $\in \{0, 1\}$.

For MNIST, we have:

$$
X = 
\begin{bmatrix}
x^{(1)}_{1} & x^{(2)}_{1} & \cdots & x^{(59999)}_{1} & x^{(60000)}_{1}\\
x^{(1)}_{2} & x^{(2)}_{2} & \cdots & x^{(59999)}_{2} & x^{(60000)}_{2}\\
\vdots & \vdots & \ddots & \vdots & \vdots \\
x^{(1)}_{783} & x^{(2)}_{783} & \cdots & x^{(59999)}_{783} & x^{(60000)}_{783}\\
x^{(1)}_{784} & x^{(2)}_{784} & \cdots & x^{(59999)}_{784} & x^{(60000)}_{784}\\
\end{bmatrix}
Y = 
\begin{bmatrix}
y^{(1)}_{1} & y^{(2)}_{1} & \cdots & y^{(59999)}_{1} & y^{(60000)}_{1}\\
y^{(1)}_{2} & y^{(2)}_{2} & \cdots & y^{(59999)}_{2} & y^{(60000)}_{2}\\
\vdots & \vdots & \ddots & \vdots & \vdots \\
y^{(1)}_{9} & y^{(2)}_{9} & \cdots & y^{(59999)}_{9} & y^{(60000)}_{9}\\
y^{(1)}_{10} & y^{(2)}_{10} & \cdots & y^{(59999)}_{10} & y^{(60000)}_{10}\\
\end{bmatrix}
$$

For MNIST when we are working with just two classes:

$$
X = 
\begin{bmatrix}
x^{(1)}_{1} & x^{(2)}_{1} & \cdots & x^{(59999)}_{1} & x^{(60000)}_{1}\\
x^{(1)}_{2} & x^{(2)}_{2} & \cdots & x^{(59999)}_{2} & x^{(60000)}_{2}\\
\vdots & \vdots & \ddots & \vdots & \vdots \\
x^{(1)}_{783} & x^{(2)}_{783} & \cdots & x^{(59999)}_{783} & x^{(60000)}_{783}\\
x^{(1)}_{784} & x^{(2)}_{784} & \cdots & x^{(59999)}_{784} & x^{(60000)}_{784}\\
\end{bmatrix}
Y = 
\begin{bmatrix}
y^{(1)} &
y^{(2)} &
\cdots &
y^{(59999)} &
y^{(60000)}
\end{bmatrix}
$$

## Let's create some random data of the correct shape

In [3]:
m = 10000
nx = 100
ny = 4

X = torch.randn(nx, m)
Y = torch.randint(2, size=(ny, m))

print("Shape of X:", X.shape)
print("Shape of Y:", Y.shape)
print("All outputs for the first five examples\n", Y[:, :5])

Shape of X: torch.Size([100, 10000])
Shape of Y: torch.Size([4, 10000])
All outputs for the first five examples
 tensor([[1, 0, 0, 1, 1],
        [0, 0, 1, 1, 1],
        [1, 1, 0, 1, 0],
        [1, 0, 0, 0, 0]])


## Creating parameters for a single layer

Each layer of the neural network has $n_l$ neruons. Each of these neurons are connected to each of the $n_{l-1}$ neruons from the previous layer. These connections are referred to as "weights," since they dictate how each value from the previous layer is weighed in this layers calculations.

$$
W^{[l]} = 
\begin{bmatrix}
w^{[l]}_{1,1} & w^{[l]}_{1,2} & \cdots & w^{[l]}_{1,n_{l-1}-1} & w^{[l]}_{1,n_{l-1}}\\
w^{[l]}_{2,1} & w^{[l]}_{2,2} & \cdots & w^{[l]}_{2,n_{l-1}-1} & w^{[l]}_{2,n_{l-1}}\\
\vdots & \vdots & \ddots & \vdots & \vdots \\
w^{[l]}_{n_l-1,1} & w^{[l]}_{n_l-1,2} & \cdots & w^{[l]}_{n_l-1,n_{l-1}-1} & w^{[l]}_{n_l-1,n_{l-1}}\\
w^{[l]}_{n_l,1} & w^{[l]}_{n_l,2} & \cdots & w^{[l]}_{n_l,n_{l-1}-1} & w^{[l]}_{n_l,n_{l-1}}\\
\end{bmatrix}
$$

Each weight matrix is $(n_l, n_{l-1})$ in shape. The first row includes all weights connected to the top neuron, the second row includes all weights connected to the second neuron, and so on.

Along with weights, we have a bias term for each neuron.

$$
b^{[l]} =
\begin{bmatrix}
b^{[l]}_{1} \\
b^{[l]}_{2} \\
\vdots \\
b^{[l]}_{n_l-1} \\
b^{[l]}_{n_l} \\
\end{bmatrix}
$$

Bias is a column vector of size $(n_l, 1)$.

Note that we do not have the parenthesis superscript as we do for $X$ and $Y$; that is because the same network weights are applied to all inputs.

## Let's randomly initialize the parameters (weights and biases)

In [4]:
def initialize_parameters(
    n0: int, n1: int, n2: int, scale: float
) -> Tuple[Tensor, Tensor, Tensor, Tensor]:
    """Initialize parameters for a 3-layer neural network.

    Args:
        n0 (int): Number of input features (aka nx)
        n1 (int): Number of neurons in layer 1
        n2 (int): Number of output neurons
        scale (float): Scaling factor for parameters

    Returns:
        Tuple[Tensor, Tensor, Tensor, Tensor]: weights and biases for 2 layers
    """
    W1 = torch.randn(n1, n0) * scale
    b1 = torch.zeros(n1, 1)
    W2 = torch.randn(n2, n1) * scale
    b2 = torch.zeros(n2, 1)
    return W1, b1, W2, b2


n0 = nx
n1 = torch.randint(40, (1, 1)).item()
n2 = ny
parameter_scale = torch.rand(1, 1).item()

W1, b1, W2, b2 = initialize_parameters(n0, n1, n2, parameter_scale)

print(f"n0: {n0}, n1: {n1}, n2: {n2}")
print("  W1", (n1, n0), ":", W1.shape)
print("  b1", (n1, 1), ":", b1.shape)
print("  W2", (n2, n1), ":", W2.shape)
print("  b2", (n2, 1), ":", b2.shape)

n0: 100, n1: 12, n2: 4
  W1 (12, 100) : torch.Size([12, 100])
  b1 (12, 1) : torch.Size([12, 1])
  W2 (4, 12) : torch.Size([4, 12])
  b2 (4, 1) : torch.Size([4, 1])


## Forward Propagation

The next step is to compute the output of the neural network. The output can go by a few names:

- prediction (or pred)
- $A^{[L]}$ (output/activation of the final layer)
- $\hat Y$

Above, $L$, refers to the index of the final layer. For our neural network, we refer to $X$ as $A^{[0]}$ to denote that it is the values at layer zero in the network.

Here are the two equations we need to implement for each layer:

$$
\begin{align}
Z^{[l]} &= W^{[l]} A^{[l-1]} + b^{[l]}\\
A^{[l]} &= g(Z^{[l]})
\end{align}
$$

where $g(\cdot)$ is any activation function. Note that these two equations compute the linear and non-linear parts of all neurons in layer $l$.

Let's look at these in more detail:

$$
Z^{[l]} = 
\begin{bmatrix}
w^{[l]}_{1,1} & w^{[l]}_{1,2} & \cdots & w^{[l]}_{1,n_{l-1}}\\
w^{[l]}_{2,1} & w^{[l]}_{2,2} & \cdots & w^{[l]}_{2,n_{l-1}}\\
\vdots & \vdots & \ddots & \vdots \\
w^{[l]}_{n_l,1} & w^{[l]}_{n_l,2} & \cdots & w^{[l]}_{n_l,n_{l-1}}\\
\end{bmatrix}
\begin{bmatrix}
a^{(1)[l-1]}_{1} & a^{(2)[l-1]}_{1} & \cdots & a^{(m)[l-1]}_{1}\\
a^{(1)[l-1]}_{2} & a^{(2)[l-1]}_{2} & \cdots & a^{(m)[l-1]}_{2}\\
\vdots & \vdots & \ddots & \vdots \\
a^{(1)[l-1]}_{n_{l-1}} & a^{(2)[l-1]}_{n_{l-1}} & \cdots & a^{(m)[l-1]}_{n_{l-1}}\\
\end{bmatrix}
+
\begin{bmatrix}
b^{[l]}_{1} \\
b^{[l]}_{2} \\
\vdots \\
b^{[l]}_{n_l} \\
\end{bmatrix}
$$

Here are the shapes: $(n_l, m) = (n_l, n_{l-1}) \cdot (n_{l-1}, m) + (n_l, 1)$

The bias vector is [broadcast](https://pytorch.org/docs/stable/notes/broadcasting.html) to a shape of $(n_l, m)$ so that it can be added to the result of the matrix multiplication.

Here is $Z^{[l]}$ after the multiplication and addition:

$$
Z^{[l]} = 
\begin{bmatrix}
w^{[l]}_{1} \cdot a^{(1)[l-1]} + b^{[l]}_{1} & w^{[l]}_{1} \cdot a^{(2)[l-1]} + b^{[l]}_{1} & \cdots & w^{[l]}_{1} \cdot a^{(m-1)[l-1]} + b^{[l]}_{1} & w^{[l]}_{1} \cdot a^{(m)[l-1]} + b^{[l]}_{1}\\
w^{[l]}_{2} \cdot a^{(1)[l-1]} + b^{[l]}_{2} & w^{[l]}_{2} \cdot a^{(2)[l-1]} + b^{[l]}_{2} & \cdots & w^{[l]}_{2} \cdot a^{(m-1)[l-1]} + b^{[l]}_{2} & w^{[l]}_{2} \cdot a^{(m)[l-1]} + b^{[l]}_{2}\\
\vdots & \vdots & \ddots & \vdots & \vdots \\
w^{[l]}_{n_l-1} \cdot a^{(1)[l-1]} + b^{[l]}_{n_l-1} & w^{[l]}_{n_l-1} \cdot a^{(2)[l-1]} + b^{[l]}_{n_l-1} & \cdots & w^{[l]}_{n_l-1} \cdot a^{(m-1)[l-1]} + b^{[l]}_{n_l-1} & w^{[l]}_{n_l-1} \cdot a^{(m)[l-1]} + b^{[l]}_{n_l-1}\\
w^{[l]}_{n_l} \cdot a^{(1)[l-1]} + b^{[l]}_{n_l} & w^{[l]}_{n_l} \cdot a^{(2)[l-1]} + b^{[l]}_{n_l} & \cdots & w^{[l]}_{n_l} \cdot a^{(m-1)[l-1]} + b^{[l]}_{n_l} & w^{[l]}_{n_l} \cdot a^{(m)[l-1]} + b^{[l]}_{n_l}\\
\end{bmatrix}
$$

In the equation above, don't let it go unnoticed that each cell (for example, $w^{[l]}_{1} \cdot a^{(1)[l-1]} + b^{[l]}_{1}$) includes a dot product of $n_{l-1}$ values.

All that's left is to apply the activation function to each value.

$$
A^{[l]} = 
\begin{bmatrix}
g(w^{[l]}_{1} \cdot a^{(1)[l-1]} + b^{[l]}_{1}) & g(w^{[l]}_{1} \cdot a^{(2)[l-1]} + b^{[l]}_{1}) & \cdots & g(w^{[l]}_{1} \cdot a^{(m-1)[l-1]} + b^{[l]}_{1}) & g(w^{[l]}_{1} \cdot a^{(m)[l-1]} + b^{[l]}_{1})\\
g(w^{[l]}_{2} \cdot a^{(1)[l-1]} + b^{[l]}_{2}) & g(w^{[l]}_{2} \cdot a^{(2)[l-1]} + b^{[l]}_{2}) & \cdots & g(w^{[l]}_{2} \cdot a^{(m-1)[l-1]} + b^{[l]}_{2}) & g(w^{[l]}_{2} \cdot a^{(m)[l-1]} + b^{[l]}_{2})\\
\vdots & \vdots & \ddots & \vdots & \vdots \\
g(w^{[l]}_{n_l-1} \cdot a^{(1)[l-1]} + b^{[l]}_{n_l-1}) & g(w^{[l]}_{n_l-1} \cdot a^{(2)[l-1]} + b^{[l]}_{n_l-1}) & \cdots & g(w^{[l]}_{n_l-1} \cdot a^{(m-1)[l-1]} + b^{[l]}_{n_l-1}) & g(w^{[l]}_{n_l-1} \cdot a^{(m)[l-1]} + b^{[l]}_{n_l-1})\\
g(w^{[l]}_{n_l} \cdot a^{(1)[l-1]} + b^{[l]}_{n_l}) & g(w^{[l]}_{n_l} \cdot a^{(2)[l-1]} + b^{[l]}_{n_l}) & \cdots & g(w^{[l]}_{n_l} \cdot a^{(m-1)[l-1]} + b^{[l]}_{n_l}) & g(w^{[l]}_{n_l} \cdot a^{(m)[l-1]} + b^{[l]}_{n_l})\\
\end{bmatrix}
$$

In [5]:
def forward_propagation(
    A0: Tensor, W1: Tensor, b1: Tensor, W2: Tensor, b2: Tensor
) -> Tuple[Tensor, Tensor]:
    """Compute the output of a 3-layer neural network.

    Args:
        A0 (Tensor): (n0, m) input matrix (aka X)
        W1 (Tensor): (n1, n0) weight matrix
        b1 (Tensor): (n1, 1) bias matrix)
        W2 (Tensor): (n2, n1) weight matrix)
        b2 (Tensor): (n2, 1) bias matrix

    Returns:
        Tuple[Tensor, Tensor]: outputs for layers 1 (n1, m) and 2 (n2, m)
    """
    Z1 = W1 @ A0 + b1
    A1 = torch.sigmoid(Z1)
    Z2 = W2 @ A1 + b2
    A2 = torch.sigmoid(Z2)
    return A1, A2


A1, A2 = forward_propagation(X, W1, b1, W2, b2)

print(f"m: {m}, n1: {n1}, n2: {n2}")
print("  A1", (n1, m), ":", A1.shape)
print("  A2", (n2, m), ":", A2.shape, Y.shape)
A2[: min(n2, 5), : min(m, 5)]

m: 10000, n1: 12, n2: 4
  A1 (12, 10000) : torch.Size([12, 10000])
  A2 (4, 10000) : torch.Size([4, 10000]) torch.Size([4, 10000])


tensor([[0.1187, 0.8149, 0.0401, 0.6765, 0.6688],
        [0.4339, 0.2414, 0.1625, 0.8435, 0.9407],
        [0.8535, 0.4525, 0.7716, 0.5258, 0.8212],
        [0.0522, 0.0170, 0.2146, 0.3959, 0.1363]])

Looking above, we can see the following:

- The sizes of $A^{[2]}$ and $Y$ are equal.
- Values in $A^{[2]} \in [0, 1]$ (range from $0$ to $1$) whereas $Y \in \{0, 1\}$ (the set of $0$ and $1$).

So, let's add a function that rounds these values to the nearest guess.

In [6]:
def get_predictions_sigmoid(
    A0: Tensor, W1: Tensor, b1: Tensor, W2: Tensor, b2: Tensor
) -> Tensor:
    """Convert the output of a sigmoid to zeros and ones.

    Args:
        A0 (Tensor): (n0, m) input matrix (aka X)
        W1 (Tensor): (n1, n0) weight matrix
        b1 (Tensor): (n1, 1) bias matrix)
        W2 (Tensor): (n2, n1) weight matrix)
        b2 (Tensor): (n2, 1) bias matrix

    Returns:
        Tensor: binary predictions of a 3-layer neural network
    """
    _, A2 = forward_propagation(A0, W1, b1, W2, b2)
    return A2.round()

Yhat = get_predictions_sigmoid(X, W1, b1, W2, b2)

print("A2\n", A2[:min(n2,5), :min(m, 5)])
print("Y\n", Y[:min(n2,5), :min(m, 5)])
print("Predictions\n", Yhat[:min(n2,5), :min(m, 5)])

A2
 tensor([[0.1187, 0.8149, 0.0401, 0.6765, 0.6688],
        [0.4339, 0.2414, 0.1625, 0.8435, 0.9407],
        [0.8535, 0.4525, 0.7716, 0.5258, 0.8212],
        [0.0522, 0.0170, 0.2146, 0.3959, 0.1363]])
Y
 tensor([[1, 0, 0, 1, 1],
        [0, 0, 1, 1, 1],
        [1, 1, 0, 1, 0],
        [1, 0, 0, 0, 0]])
Predictions
 tensor([[0., 1., 0., 1., 1.],
        [0., 0., 0., 1., 1.],
        [1., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0.]])


We can now compute an output, but the output is currently random. We probably have about a 50% accuracy. Let's check:

In [7]:
error = (Y - Yhat) # Get the difference
absolute_error = error.abs() # Ignore the sign
mean_error = absolute_error.mean() # Get the average
accuracy = 1 - mean_error # This works since the output is either 0 or 1

accuracy, 1 - (Y - Yhat).abs().mean()

(tensor(0.4965), tensor(0.4965))

## Now we need learn the parameters

To make the error zero, we need some way to make $\hat Y$ converge toward $Y$. (Note, we cannot always make the error exaclty zero, but we can often get close.)

We cannot change $Y$ (it wouldn't make sense to relabel a cat as a dog), but we can change $\hat Y$ by adjusting the parameters $W^{[l]}$ and $b^{[l]}$.

But what is the best way to minimize them? In deep learning we use gradient descent. That is, we compute the gradient of an objective function with respect to the parameters we can change. This tells us *how* we should change the parameters (make them bigger or smaller).

As we've discussed in class, we can use any of a number of objective functions. But let's focus on binary cross entropy loss so that we have a specific example:

$$
\mathcal{L}(\hat Y, Y) = -( Y \log(\hat Y) + (1 - Y) \log(1 - \hat Y))
$$

Now we just need to compute some derivatives. Namely (for the case of a 3-layer network), $\frac{\partial \mathcal{L}}{\partial W^{[1]}}$, $\frac{\partial \mathcal{L}}{\partial b^{[1]}}$, $\frac{\partial \mathcal{L}}{\partial W^{[2]}}$, and $\frac{\partial \mathcal{L}}{\partial b^{[2]}}$.

Starting with $W^{[2]}$ and applying the chain rule, we end up with this:

$$
\frac{\partial \mathcal{L}}{\partial W^{[2]}} = 
\color{blue}{\frac{\partial \mathcal{L}}{\partial \hat Y}} \cdot
\color{green}{\frac{\partial \hat Y}{\partial Z^{[2]}}} \cdot
\color{magenta}{\frac{\partial Z^{[2]}}{\partial W^{[2]}}}
$$

We can compute each of these separately.

$$
\begin{align}
\color{blue}{\frac{\partial \mathcal{L}}{\partial \hat Y}}
&= -\frac{\partial}{\partial \hat Y}( Y \log(\hat Y) + (1 - Y) \log(1 - \hat Y))\\
&= -(\frac{Y}{\hat Y} + \frac{1-Y}{1-\hat Y}\cdot \frac{\partial 1 - \hat Y}{\partial \hat Y})\\
&= -\frac{Y}{\hat Y} + \frac{1-Y}{1-\hat Y}
\end{align}
$$

Now let's compute the second component.

$$
\begin{align}
\color{green}{\frac{\partial \hat Y}{\partial Z^{[2]}}}
&= \frac{\partial}{\partial Z^{[2]}} g(Z^{[2]})\\
&= g'(Z^{[2]})
\end{align}
$$

Here, $g'(\cdot)$ will depend on the activation function. For the sigmoid function $\sigma(\cdot)$ it is $\sigma'(x) = \sigma(x)(1 - \sigma(x))$. Now the final component:

$$
\begin{align}
\color{magenta}{\frac{\partial Z^{[2]}}{\partial W^{[2]}}}
&= \frac{\partial}{\partial W^{[2]}} W^{[2]} A^{[1]} + b^{[2]}\\
&= A^{[1]}
\end{align}
$$

Putting this all together we have:

$$
\frac{\partial \mathcal{L}}{\partial W^{[2]}} = 
\color{blue}{-\frac{Y}{\hat Y} + \frac{1-Y}{1-\hat Y}} \cdot
\color{green}{g'(Z^{[2]})} \cdot
\color{magenta}{A^{[1]}}
$$

For this example, let's assume we are using the sigmoid function. Simpligying and rearranging this equation so that the dimension match, we take into account that we should use the average over all $m$ examples, and the correct terms are computed, we have:

$$
\frac{\partial \mathcal{L}}{\partial W^{[2]}}
= \frac{1}{m} (\hat Y - Y) A^{[1]T}
$$

Next up, we need to perform a similar procedure for $\frac{\partial \mathcal{L}}{\partial b^{[1]}}$. You might notice that this term also includes $\frac{\partial \mathcal{L}}{\partial Z^{[2]}}$, which we've already computed as $(\hat Y - Y)$.

$$
\begin{align}
\frac{\partial \mathcal{L}}{\partial b^{[1]}} 
&= dZ^{[2]} \cdot \frac{\partial Z^{[2]}}{\partial b^{[2]}}\\
&= (\hat Y - Y) \cdot \frac{\partial}{\partial b^{[2]}} W^{[2]} A^{[1]} + b^{[2]}\\
&= (\hat Y - Y)
\end{align}
$$

But again, we want to take the average across all $m$ examples:

$$
\frac{\partial \mathcal{L}}{\partial b^{[1]}} 
= \frac{1}{m} \sum_{i=1}^m (\hat Y - Y)
$$

Note: averaging over all $m$ examples is done implicitly by the matrix multiplications when computing the derivative of $W^{[2]}$.

Now we are left with the paramters from the first layer. You'll start to notice patterns forming here.

$$
\frac{\partial \mathcal{L}}{\partial W^{[1]}} = 
\color{blue}{\frac{\partial \mathcal{L}}{\partial \hat Y}} \cdot
\color{green}{\frac{\partial \hat Y}{\partial Z^{[2]}}} \cdot
\color{magenta}{\frac{\partial Z^{[2]}}{\partial A^{[1]}}} \cdot
\color{olive}{\frac{\partial A^{[1]}}{\partial Z^{[1]}}} \cdot
\color{brown}{\frac{\partial Z^{[1]}}{\partial W^{[1]}}}
$$

We already have the results for the first two terms. These will be propagated backward from layer 2 to layer 1. The third and final terms are new, but the second to last term is also something we've already seen (it depends on the ativation function).

$$
\begin{align}
\color{magenta}{\frac{\partial Z^{[2]}}{\partial A^{[1]}}} 
&= \frac{\partial}{\partial A^{[1]}} W^{[2]} A^{[1]} + b^{[2]}\\
&= W^{[2]}
\end{align}
$$


$$
\begin{align}
\color{brown}{\frac{\partial Z^{[1]}}{\partial W^{[1]}}}
&= \frac{\partial}{\partial W^{[1]}} W^{[1]} A^{[0]} + b^{[1]}\\
&= A^{[0]}
\end{align}
$$

We again need to consider averaging the gradient for all $m$ examples and a use a specific activation funcion (again a sigmoid):

$$
\frac{\partial \mathcal{L}}{\partial W^{[1]}} = 
\frac{1}{m} W^{[2]T} (\hat Y - Y) \cdot (A^{[1]} \cdot (1 - A^{[1]})) A^{[0]T}\\
$$

Looking at the shapes in the expession above, we have:

$$
(n1, n0) = ((n1, n2) (n2, m) \times (n1, m)) (m, n0)
$$

which results in the same shape as $W^{[1]}$.

One last equation:

$$
\frac{\partial \mathcal{L}}{\partial b^{[1]}} = 
\color{blue}{\frac{\partial \mathcal{L}}{\partial \hat Y}} \cdot
\color{green}{\frac{\partial \hat Y}{\partial Z^{[2]}}} \cdot
\color{magenta}{\frac{\partial Z^{[2]}}{\partial A^{[1]}}} \cdot
\color{olive}{\frac{\partial A^{[1]}}{\partial Z^{[1]}}} \cdot
\color{brown}{\frac{\partial Z^{[1]}}{\partial b^{[1]}}}
$$

The only new term is the last one:

$$
\begin{align}
\color{brown}{\frac{\partial Z^{[1]}}{\partial b^{[1]}}}
&= \frac{\partial}{\partial b^{[1]}} W^{[1]} A^{[0]} + b^{[1]}\\
&= 1
\end{align}
$$

After rearranging:

$$
\frac{\partial \mathcal{L}}{\partial b^{[1]}} = 
\frac{1}{m} \sum_{i=1}^m W^{[2]T} (\hat Y - Y) \cdot (A^{[1]} \cdot (1 - A^{[1]}))\\
$$

# Summary of equations

I've placed the sizes of each matrix in place.

$$
\begin{align}
dZ^{[2]} \color{gray}{(n_2, m)} &= \hat Y - Y\\
dW^{[2]} \color{gray}{(n_2, n_1)} &= \frac{1}{m} dZ^{[2]} A^{[1]T}\\
db^{[2]} \color{gray}{(n_2, 1)} &= \frac{1}{m} \sum_{i=1}^m dZ^{[2]}\\
dZ^{[1]} \color{gray}{(n_1, m)} &= W^{[2]T} dZ^{[2]} \cdot A^{[1]} \cdot (1 - A^{[1]})\\
dW^{[1]} \color{gray}{(n_1, n_0)} &= \frac{1}{m} dZ^{[1]} X^T\\
db^{[1]} \color{gray}{(n_1, 1)} &= \frac{1}{m} \sum_{i=1}^m dZ^{[1]}\\
\end{align}
$$

where $X=A^{[0]}$ and $Y=A^{[2]}$.

In [8]:
def backward_propagation(
    A0: Tensor, A1: Tensor, A2: Tensor, Y: Tensor, W2: Tensor
) -> Tuple[Tensor, Tensor, Tensor, Tensor]:
    """Compute gradients of a 3-layer neural network's parameters.

    Args:
        A0 (Tensor): (n0, m) input matrix (aka X)
        A1 (Tensor): (n1, m) output of layer 1 from forward propagation
        A2 (Tensor): (n2, m) output of layer 2 from forward propagation
        Y (Tensor): (n2, m) correct targets (aka labels)
        W2 (Tensor): (n2, n1) weight matrix)

    Returns:
        Tuple[Tensor, Tensor, Tensor, Tensor]: gradients for weights and biases
    """
    m = Y.shape[1]

    dZ2 = A2 - Y
    dW2 = (1 / m) * (dZ2 @ A1.T)
    db2 = (1 / m) * dZ2.sum(dim=1, keepdim=True)

    dZ1 = (W2.T @ dZ2) * (A1 * (1 - A1))
    dW1 = (1 / m) * (dZ1 @ A0.T)
    db1 = (1 / m) * dZ1.sum(dim=1, keepdim=True)
    return dW1, db1, dW2, db2


dW1, db1, dW2, db2 = backward_propagation(X, A1, A2, Y, W2)
print("dW1", (n1, n0), ":", dW1.shape)
print("db1", (n1, 1), ":", db1.shape)
print("dW2", (n2, n1), ":", dW2.shape)
print("db2", (n2, 1), ":", db2.shape)

dW1 (12, 100) : torch.Size([12, 100])
db1 (12, 1) : torch.Size([12, 1])
dW2 (4, 12) : torch.Size([4, 12])
db2 (4, 1) : torch.Size([4, 1])


## Updating paramters

The next step is to update the parameters. However, we don't want to overshoot the global minimum. So, we add an extra parameter, the *learning rate*.

In [9]:
def update_parameters(
    W1: Tensor,
    b1: Tensor,
    W2: Tensor,
    b2: Tensor,
    dW1: Tensor,
    db1: Tensor,
    dW2: Tensor,
    db2: Tensor,
    lr: float,
) -> Tuple[Tensor, Tensor, Tensor, Tensor]:
    """Update parameters of a 3-layer neural network.

    Args:
        W1 (Tensor): (n1, n0) weight matrix
        b1 (Tensor): (n1, 1) bias matrix)
        W2 (Tensor): (n2, n1) weight matrix)
        b2 (Tensor): (n2, 1) bias matrix
        dW1 (Tensor): (n1, n0) gradient matrix
        db1 (Tensor): (n1, 1) gradient matrix)
        dW2 (Tensor): (n2, n1) gradient matrix)
        db2 (Tensor): (n2, 1) gradient matrix
        lr (float): learning rate

    Returns:
        Tuple[Tensor, Tensor, Tensor, Tensor]: updated network parameters
    """
    W1 = W1 - lr * dW1
    b1 = b1 - lr * db1
    W2 = W2 - lr * dW2
    b2 = b2 - lr * db2
    return W1, b1, W2, b2


learning_rate = 0.01
W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

print("W1", (n1, n0), ":", W1.shape)
print("b1", (n1, 1), ":", b1.shape)
print("W2", (n2, n1), ":", W2.shape)
print("b2", (n2, 1), ":", b2.shape)

W1 (12, 100) : torch.Size([12, 100])
b1 (12, 1) : torch.Size([12, 1])
W2 (4, 12) : torch.Size([4, 12])
b2 (4, 1) : torch.Size([4, 1])


## Computing cost

To check our progress, we can compute the cost after each training epoch. We expect the cost to go down, which would indicate learning. We will have a **loss** value for each output for each example. **Cost** is the average loss value across all examples.

In [10]:
def compute_cost(A2: Tensor, Y: Tensor) -> Tensor:
    """Compute cost using binary cross entropy loss.

    Args:
        A2 (Tensor): (n2, m) matrix of neural network output values
        Y (Tensor): (n2, m) correct targets (aka labels)

    Returns:
        float: computed cost
    """
    m = Y.shape[1]
    losses = -(Y * torch.log(A2) + (1 - Y) * torch.log(1 - A2))
    cost = (1 / m) * losses.sum(dim=1, keepdims=True)
    return cost


cost = compute_cost(A2, Y)
print("cost", (n2, 1), ":", cost.shape)

cost (4, 1) : torch.Size([4, 1])


In [11]:
def learn(
    X: Tensor,
    Y: Tensor,
    num_hidden: int,
    param_scale: float,
    num_epochs: int,
    learning_rate: float,
) -> Tuple[Tensor, Tensor, Tensor, Tensor]:
    """A function for performing batch gradient descent.

    Args:
        X (Tensor): (nx, m) matrix of input features
        Y (Tensor): (n2, m) matrix of correct targets (aka labels)
        num_hidden (int): number of neurons in layer 1
        param_scale (float): scaling factor for initializing parameters
        num_epochs (int): number of training passes through all data
        learning_rate (float): learning rate

    Returns:
        Tuple[Tensor, Tensor, Tensor, Tensor]: parameters of a 3-layer neural network
    """
    n0 = X.shape[0]
    n1 = num_hidden
    n2 = Y.shape[0]

    W1, b1, W2, b2 = initialize_parameters(n0, n1, n2, param_scale)

    for epoch in range(num_epochs):

        A1, A2 = forward_propagation(X, W1, b1, W2, b2)

        cost = compute_cost(A2, Y)

        dW1, db1, dW2, db2 = backward_propagation(X, A1, A2, Y, W2)

        W1, b1, W2, b2 = update_parameters(
            W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate
        )

        print(f"{epoch + 1:>3}/{num_epochs}: Cost={cost.mean():.2f}")
        
    return W1, b1, W2, b2

num_epochs = 8
learn(X, Y, n1, parameter_scale, num_epochs, learning_rate);

  1/8: Cost=1.26
  2/8: Cost=1.25
  3/8: Cost=1.24
  4/8: Cost=1.23
  5/8: Cost=1.23
  6/8: Cost=1.22
  7/8: Cost=1.21
  8/8: Cost=1.20
